<a href="https://colab.research.google.com/github/rutkovskii/FlightAnalysis/blob/usingBF4/RusAirtraffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [210]:
# Imports
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import re

Basically, I can exclude bs4, because I get Country names from "flugzeuginfo"

# Exctracting Data

###Working with Flight Data

In [211]:
# Read all flights for January and February 2022

# Source: https://zenodo.org/record/6325961#.YkHzUzdBxQI

flights_jan = pd.read_csv("/content/flightlist_20220101_20220131.csv")
flights_feb = pd.read_csv("/content/flightlist_20220201_20220228.csv")

In [212]:
# Sneakpeak
flights_jan.head()

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,QTR859,QR859,06a13c,A7-BAZ,B77W,OTHH,OTHH,2021-12-31 00:11:49+00:00,2022-01-01 02:10:29+00:00,2022-01-01 00:00:00+00:00,25.289062,51.612142,0.0,25.249096,51.629575,15.24
1,CAO1023,NaN,780bb7,B-2096,B77L,KORD,EHAM,2021-12-31 01:29:29+00:00,2022-01-01 13:23:35+00:00,2022-01-01 00:00:00+00:00,41.969030,-87.929524,304.8,52.351669,4.710938,-99.06
2,ETH724,NaN,040141,NaN,NaN,KEWR,LOWW,2021-12-31 03:26:20+00:00,2022-01-01 04:47:19+00:00,2022-01-01 00:00:00+00:00,40.688141,-74.171326,0.0,48.113159,16.563126,99.06
3,SEJ241,NaN,800d1a,NaN,NaN,VECC,VECC,2021-12-31 04:27:41+00:00,2022-01-01 03:03:58+00:00,2022-01-01 00:00:00+00:00,22.709764,88.453217,304.8,22.588852,88.440091,259.08
4,QFA7550,NaN,abb6dc,N854GT,B748,KORD,YSSY,2021-12-31 04:34:40+00:00,2022-01-01 01:30:28+00:00,2022-01-01 00:00:00+00:00,41.984067,-87.942682,304.8,-33.965630,151.181087,22.86


In [213]:
# Attempt to find Pulkovo Airport in this data set 
# Origins and Destinations are written in ICAO codes
# ICAO — International Civil Aviation Organization

# Pulkovo's ICAO code: ULLI
flights_jan.loc[flights_jan['origin'] == "ULLI"].head(10)


,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
189344,SDM6115,NaN,4248e4,VQ-BAV,A319,ULLI,NaN,2022-01-03 16:39:52+00:00,2022-01-03 17:59:58+00:00,2022-01-03 00:00:00+00:00,59.780848,30.342818,609.6,52.193298,39.805350,10683.24
189664,PBD6822,NaN,42442e,NaN,NaN,ULLI,UUEE,2022-01-03 16:43:55+00:00,2022-01-03 17:40:19+00:00,2022-01-03 00:00:00+00:00,59.781034,30.340829,609.6,55.957233,37.320471,510.54
189843,DLH1437,LH1437,3c6758,D-AIZX,A320,ULLI,EDDF,2022-01-03 16:46:23+00:00,2022-01-03 19:25:24+00:00,2022-01-03 00:00:00+00:00,59.782562,30.334808,914.4,50.042643,8.560354,137.16
190716,FIN706,NaN,461f35,OH-LVH,A319,ULLI,EFHK,2022-01-03 16:57:51+00:00,2022-01-03 17:35:57+00:00,2022-01-03 00:00:00+00:00,59.781052,30.339844,914.4,60.315128,24.944752,205.74
191305,LOT686,NaN,48adab,NaN,NaN,ULLI,EPWA,2022-01-03 17:05:00+00:00,2022-01-03 18:43:10+00:00,2022-01-03 00:00:00+00:00,59.781326,30.339661,914.4,52.150563,20.980198,220.98
191449,SWR1311,LX1311,4b1800,HB-JCI,NaN,ULLI,LSZH,2022-01-03 17:06:40+00:00,2022-01-03 19:56:39+00:00,2022-01-03 00:00:00+00:00,59.783798,30.322174,914.4,47.476951,8.543067,411.48
192201,SDM6023,NaN,4242e9,VP-BQK,A319,ULLI,UUWW,2022-01-03 17:16:19+00:00,2022-01-03 18:16:29+00:00,2022-01-03 00:00:00+00:00,59.782013,30.332153,914.4,55.570496,37.168080,609.60
192766,AFL025,NaN,424350,VP-BFQ,A321,ULLI,UUEE,2022-01-03 17:23:17+00:00,2022-01-03 18:18:18+00:00,2022-01-03 00:00:00+00:00,59.782803,30.327948,914.4,55.950529,37.276011,716.28
193072,AUL581,NaN,424486,NaN,NaN,ULLI,NaN,2022-01-03 17:27:01+00:00,2022-01-03 19:03:58+00:00,2022-01-03 00:00:00+00:00,59.779451,30.352762,609.6,53.526627,49.574629,2590.80
193769,AUL517,NaN,4243af,NaN,NaN,ULLI,NaN,2022-01-03 17:36:20+00:00,2022-01-03 18:22:55+00:00,2022-01-03 00:00:00+00:00,59.781173,30.339787,914.4,64.748064,32.028351,11582.40


### Extracting International Airports and the Page 

In [214]:
# Getting tables with all international airports
url = "https://en.wikipedia.org/wiki/List_of_international_airports_by_country"
airport_dfs = pd.read_html(url)


In [215]:
# Getting names of the countries

url = "https://en.wikipedia.org/wiki/List_of_international_airports_by_country"
response=requests.get(url)
print(response.status_code)


200


### Getting DFs with IATA and ICAO Codes

In [216]:
# Getting codes
url = "http://www.flugzeuginfo.net/table_airportcodes_country-location_en.php"
iata_icao_dfs = pd.read_html(url)

len(iata_icao_dfs)
iata_icao_dfs[0]


,IATA,ICAO,Location,Airport,Country
0,BIM,OABN,Bamyan,Bamyan,Afghanistan
1,BST,OABT,Bost,Bost,Afghanistan
2,BPM,OAIX,Charikar,Bagram Air Base,Afghanistan
3,CCN,OACC,Charkhcharan,Charkhcharan,Afghanistan
4,FBD,OAFZ,Faizabad,Faizabad,Afghanistan
...,...,...,...,...,...
302,KLU,LOWK,Klagenfurt,Klagenfurt,Austria
303,SZG,LOWS,Salzburg,Salzburg,Austria
304,VIE,LOWW,Wien (Vienna),Wien - Schwechat,Austria
305,BAK,UBBB,Baku,Bina,Azerbaijan


### Getting Names and ICAO codes of Russian Airports

In [217]:
# Getting tables with all Russian airports

ru_airports = pd.read_html("https://en.wikipedia.org/wiki/List_of_airports_in_Russia")

# We need 1st table because it contains the relevant data
ru_airports = ru_airports[0]
ru_airports.head()

,Location,ICAO,IATA,Airport name
0,Northwestern Federal District[edit],Northwestern Federal District[edit],Northwestern Federal District[edit],Northwestern Federal District[edit]
1,Amderma,ULDD,AMV,Amderma Airport
2,Apatity,ULMK,KVK,Kirovsk-Apatity Airport (Khibiny)
3,Arkhangelsk,ULAA,ARH,Talagi Airport
4,Arkhangelsk,ULAH,NaN,Vaskovo Airport


# Cleaning Data

## Create IATA Codes
Combining tables with country names

Ps. Could have actually avoided using it because ```iata_icao_dfs``` have Country names corresponding with ICAO and IATA codes



In [218]:
soup = BeautifulSoup(response.text, 'html.parser')


# Extract all Countries before Oceania
countries = []

for level in soup.find_all('li',{'class':'toclevel-3'}):
  #hit = level.text.strip()
  country = level.find('span',{'class':'toctext'}).text.strip()
  countries.append(country)

print(len(countries))


# Extract Oceania countries
t1 = soup.find('li',{'class':"toclevel-1 tocsection-234"})

for level in t1.find_all('li',{'class':'toclevel-2'}):
  country = level.find('span',{'class':'toctext'}).text.strip()
  countries.append(country)

print(len(countries))

208
234


In [219]:
# Remove table that is not the airport table
print(len(airport_dfs))
if len(airport_dfs) > 234:
  airport_dfs.pop(0)
print(len(airport_dfs))


modified_airport_dfs = []

# Cleaning Columns and Assigning country to the airport
i = 0 
for df in airport_dfs:
  if "IATA Code" in df.columns:
    df.rename(columns={"IATA Code": "IATA"},inplace=True)
  m_df = df[["IATA","Airport"]]

  # Assigning counttry
  m_df.insert(2, 'Country', countries[i])
  modified_airport_dfs.append(m_df)

  i+=1


# Creating 1 Massive dataframe
IATA_df = pd.concat(modified_airport_dfs)
IATA_df.reset_index(drop=True,inplace=True)
len(IATA_df)
IATA_df





235
234


,IATA,Airport,Country
0,AZR,Touat-Cheikh Sidi Mohamed Belkebir Airport,Algeria
1,ALG,Houari Boumediene Airport,Algeria
2,AAE,Rabah Bitat Airport,Algeria
3,BLJ,Batna Airport,Algeria
4,BJA,Soummam Airport,Algeria
...,...,...,...
1439,FUN,Funafuti International Airport,Tuvalu
1440,SON,Santo-Pekoa International Airport,Vanuatu
1441,VLI,Bauerfield International Airport,Vanuatu
1442,FUT,Pointe Vele Airport,Wallis and Futuna


In [ ]:
iata_icao_df = pd.concat(iata_icao_dfs)
iata_icao_df

In [220]:
codes = iata_icao_df
no_codes = IATA_df

In [225]:
codes.head()
no_codes.head()

,IATA,Airport,Country
0,AZR,Touat-Cheikh Sidi Mohamed Belkebir Airport,Algeria
1,ALG,Houari Boumediene Airport,Algeria
2,AAE,Rabah Bitat Airport,Algeria
3,BLJ,Batna Airport,Algeria
4,BJA,Soummam Airport,Algeria


In [228]:
pd.merge(no_codes, codes[['IATA','ICAO']], on='IATA')

,IATA,Airport,Country,ICAO
0,AZR,Touat-Cheikh Sidi Mohamed Belkebir Airport,Algeria,DAUA
1,ALG,Houari Boumediene Airport,Algeria,DAAG
2,AAE,Rabah Bitat Airport,Algeria,DABB
3,BJA,Soummam Airport,Algeria,DAAE
4,CZL,Mohamed Boudiaf International Airport,Algeria,DABC
...,...,...,...,...
1251,TBU,Fuaʻamotu International Airport,Tonga,NFTF
1252,VAV,Vavaʻu International Airport,Tonga,NFTV
1253,FUN,Funafuti International Airport,Tuvalu,NGFU
1254,SON,Santo-Pekoa International Airport,Vanuatu,NVSS


In [90]:
# flights_jan
flights_jan = flights_jan[["callsign","origin","destination"]]
print(len(flights_feb))

# flights22
flights_feb = flights_feb[["callsign","origin","destination"]]
print(len(flights_feb))
flights = pd.concat([flights_jan,flights_feb])
print(len(flights))


2239224
2239224
2472381


In [91]:

# Relevant Columns
ru_airports = ru_airports[["Airport name","ICAO"]]


# Dropping the rows with unused airports, with region names, and Simferopol, since it is not internation 
print("Size of df before:", len(ru_airports))

discard = ["abandoned","closed","edit","Simferopol"]
ru_airports = ru_airports[~ru_airports["Airport name"].str.contains('|'.join(discard))]

print("Size of df before:", len(ru_airports))


# To see names of the airports
#for row in ru_airports["Airport name"]:
# print(row)


Size of df before: 352
Size of df before: 323


In [99]:
# Exctracting list of all Russian Airports


letters = ["A","B","C","E","F","G","H","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","Y","Z"]

url = "https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_"

urls = []
for letter in letters:
  region_url = url+letter
  urls.append(region_url)

print(urls)

A = pd.read_html(urls[0])
A[2]


['https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_A', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_B', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_C', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_E', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_F', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_G', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_H', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_J', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_K', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_L', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_M', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_N', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_O', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_P', 'https://en.wikipedia.org/wiki/List_of_airports_by_ICAO_code:_Q', 'https://

,".mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:""[ ""}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:"" ]""}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vteLists of airports",".mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:""[ ""}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:"" ]""}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vteLists of airports.1"
0,By IATA code,A B C D E F G H I J K L M N O P Q R S T U V W ...
1,By ICAO code,A B C D E F G H I J K L M N O P Q R S T U V W ...
2,By location,By continent Africa Antarctica Asia Europe Nor...
3,Busiest airports,overall busiest air traffic passengers interna...
4,Other lists,Countries without airports Defunct internation...
